In [1]:
! pip install azure-storage-blob requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 28.5 MB/s eta 0:00:0000:01


In [ ]:
import requests as req
import pandas as pd
from azure.storage.blob import BlobServiceClient
import os
from tqdm import tqdm


In [139]:
connection_string = os.getenv('AZURE_CONNECTION_STRING')
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [141]:
def upload_blob(blob_service_client, blob_name, local_file_path):
  container_name = "bdmcontainerp1"
  container_client = blob_service_client.get_container_client(container_name)

  with open(local_file_path, "rb") as data:
      container_client.upload_blob(name=blob_name, data=data, overwrite=True)

  blobs = [blob.name for blob in container_client.list_blobs()]
  if blob_name not in blobs:
    raise Exception("Blob was not uploaded")

# Landing Zone

## Weather Data
Check for more examples:  https://open-meteo.com/


In [129]:
import os
import json
import requests as req
from datetime import datetime, timedelta

def save_json_response(city, date_str, data, directory="weather"):
    city_dir = os.path.join(directory, city)
    os.makedirs(city_dir, exist_ok=True)
    
    file_path = os.path.join(city_dir, f"{date_str}.json")
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)


start_date = datetime.strptime('2024-03-24', '%Y-%m-%d')
end_date = datetime.strptime('2024-06-25', '%Y-%m-%d')

city_coords = {
    'Barcelona': {'latitude': 41.3874, 'longitude': 2.1686},
    'Paris': {'latitude': 48.8575, 'longitude': 2.3514},
    'Madrid': {'latitude': 40.4167, 'longitude': -3.7033},
    'Rome': {'latitude': 41.8967, 'longitude': 12.4822}
}

hourly = 'temperature_2m,rain,snowfall,precipitation,cloud_cover,wind_speed_10m,sunshine_duration'
weather_endpoint = 'https://archive-api.open-meteo.com/v1/archive'

# Loop por fecha
current_date = start_date
while current_date <= end_date:
    next_date = current_date + timedelta(days=1)

    current_date_str = current_date.strftime('%Y-%m-%d')
    next_date_str = next_date.strftime('%Y-%m-%d')

    for city, coords in city_coords.items():
        latitude = coords['latitude']
        longitude = coords['longitude']

        querystring = {
            'latitude': latitude,
            'longitude': longitude,
            'hourly': hourly,
            'start_date': current_date_str,
            'end_date': current_date_str,
            'timezone': 'auto'
        }

        try:
            response = req.get(weather_endpoint, params=querystring)
            response.raise_for_status()
            data = response.json()
            save_json_response(city, current_date_str, data)
        except Exception as e:
            print(f"Error al obtener datos para {city} el {current_date_str}: {e}")

    current_date = next_date


Error al obtener datos para Rome el 2024-03-30: 504 Server Error: Gateway Time-out for url: https://archive-api.open-meteo.com/v1/archive?latitude=41.8967&longitude=12.4822&hourly=temperature_2m%2Crain%2Csnowfall%2Cprecipitation%2Ccloud_cover%2Cwind_speed_10m%2Csunshine_duration&start_date=2024-03-30&end_date=2024-03-30&timezone=auto


In [132]:
latitude = 41.8967
longitude = 12.4822
city_coords = {
                'Barcelona': {'latitude': 41.3874, 'longitude': 2.1686},
                'Paris': {'latitude': 48.8575, 'longitude': 2.3514},
                'Madrid': {'latitude': 40.4167, 'longitude': 3.7033},
                'Rome': {'latitude': 41.8967, 'longitude': 12.4822}
                }

start_date = '2024-03-30'
end_date = '2024-03-30'
hourly = 'temperature_2m,rain,snowfall,precipitation,cloud_cover,wind_speed_10m,sunshine_duration'
querystring = {
                'latitude':latitude,
                'longitude':longitude,
                'hourly':hourly,
                'start_date':start_date,
                'end_date':end_date
                }

weather_endpoint = f'https://archive-api.open-meteo.com/v1/archive'
weather_res = req.get(weather_endpoint, params=querystring).json()
save_json_response('Rome', start_date, weather_res)

## Accomodation Data

Check for more examples: https://rapidapi.com/DataCrawler/api/booking-com15/playground/apiendpoint_818c2744-8507-4071-829e-d080b667a06c

In [5]:
def save_images(res, client, querystring, city):
    hotels = res['data']['hotels']
    for hotel in hotels:
        photos = hotel['property']['photoUrls']
        for photo in photos:
            img_res = req.get(photo, stream=True)
            filename = city + '/' + str(hotel['hotel_id']) + '_' + querystring['dest_id'] + '_' + querystring['arrival_date'] + '_' + querystring['departure_date'] + '.jpg'
            if img_res.status_code == 200:
                with open('accomodation_images/' + filename, "wb") as file:
                    for chunk in img_res.iter_content(1024):  # Guardar en bloques de 1024 bytes
                        file.write(chunk)
            upload_blob(client, f'accomodation_images/{filename}', f'accomodation_images/{filename}')
            os.remove(f'accomodation_images/{filename}')

In [6]:
def save_json(res, filename):
    with open(filename, 'w') as file:
        json.dump(res, file)

In [ ]:
accomodation_endpoint = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"
weather_endpoint = f'https://api.open-meteo.com/v1/forecast'

delta = timedelta(days=1)

destination_ids = {
    "Barcelona": "-372490",
    "Rome": "-126693",
    "Madrid": "-390625",
    "Paris": "-1456928"
}

headers = {
            "x-rapidapi-key": os.environ["RAPID_API_KEY"],
            "x-rapidapi-host": os.environ["RAPID_API_HOST"]
        }

accomodation_querystring = {
            "dest_id": '',
            "search_type": "CITY",
            "arrival_date": '',
            "departure_date": '',
            "adults": "2",
            "children_age": "0",
            "room_qty": "1",
            "page_number": "1",
            "units": "metric",
            "temperature_unit": "c",
            "languagecode": "en-us",
            "currency_code": "EUR"
        }

destination_coords = {
                'Barcelona': {'latitude': 41.3874, 'longitude': 2.1686},
                'Paris': {'latitude': 48.8575, 'longitude': 2.3514},
                'Madrid': {'latitude': 40.4167, 'longitude': 3.7033},
                'Rome': {'latitude': 41.8967, 'longitude': 12.4822}
                }

weather_metrics = 'temperature_2m,precipitation_probability,apparent_temperature,precipitation,cloud_cover,wind_speed_10m'
weather_querystring = {
                'latitude': '',
                'longitude': '',
                'hourly': weather_metrics,
                'start_date': '',
                'end_date': ''
                }

start = datetime.strptime('2025-06-05', "%Y-%m-%d")
end = datetime.strptime('2025-06-10', "%Y-%m-%d")
last_city = 'Rome'
try:
    for single_date in tqdm([start + i * delta for i in range((end - start).days + 1)]):
        arrival_date = single_date.strftime("%Y-%m-%d")
        departure_date = (single_date + timedelta(days=1)).strftime("%Y-%m-%d")
        start_date = single_date.replace(year=single_date.year - 1).strftime("%Y-%m-%d")
        end_date = (single_date + timedelta(days=1)).replace(year=single_date.year - 1).strftime("%Y-%m-%d")

        for city, city_id in destination_ids.items():
            if last_city and city != last_city:
                continue
            print(city)
            accomodation_querystring['dest_id'] = city_id
            accomodation_querystring['arrival_date'] = arrival_date
            accomodation_querystring['departure_date'] = departure_date
            
            accomodation_res = req.get(accomodation_endpoint, headers=headers, params=accomodation_querystring)
            if accomodation_res.status_code == 200:
                print("Accomodation data retrieved successfully")
                accomodation_res = accomodation_res.json()
                save_images(accomodation_res, blob_service_client, accomodation_querystring, city)
                json_res_filename =  city + '/' + accomodation_querystring['arrival_date'] + '_' + accomodation_querystring['departure_date'] + '.json'
                save_json(accomodation_res, f'accomodation/{json_res_filename}')
                upload_blob(blob_service_client, f'accomodation/{json_res_filename}', f'accomodation/{json_res_filename}')
                os.remove(f'accomodation/{json_res_filename}')
            else:
                print('Accomodation', city, arrival_date, departure_date, accomodation_res)
                last_city = city
                raise

            latitude, longitude = destination_coords[city].values()
            weather_querystring['latitude'] = latitude
            weather_querystring['longitude'] = longitude
            weather_querystring['start_date'] = start_date
            weather_querystring['end_date'] = start_date
            weather_json_filename = city + '/' + start_date + '.json'
            
            weather_res = req.get(weather_endpoint, params=weather_querystring)
            if weather_res.status_code == 200:
                print("Weather data retrieved successfully")
                weather_res = weather_res.json()
                save_json(weather_res, f'weather/{weather_json_filename}')
                upload_blob(blob_service_client, f'weather/{weather_json_filename}', f'weather/{weather_json_filename}')
                os.remove(f'weather/{weather_json_filename}')
            else:
                print('Weather', city, start_date, end_date, weather_res)
                last_city = city
                raise
            
            if last_city:
                last_city = None

except Exception as e:
    print(e)
    print(last_city, arrival_date, departure_date)

  0%|          | 0/218 [00:00<?, ?it/s]

Rome
Accomodation data retrieved successfully
Weather data retrieved successfully
Madrid
Accomodation data retrieved successfully
Weather data retrieved successfully
Paris
Accomodation data retrieved successfully
Weather data retrieved successfully


  0%|          | 1/218 [14:33<52:38:21, 873.28s/it]

Barcelona
Accomodation data retrieved successfully
Weather data retrieved successfully
Rome
Accomodation data retrieved successfully
Weather data retrieved successfully
Madrid
Accomodation data retrieved successfully
Weather data retrieved successfully
Paris
Accomodation data retrieved successfully
Weather data retrieved successfully


  1%|          | 2/218 [32:57<60:33:24, 1009.28s/it]

...

Barcelona
Accomodation data retrieved successfully
Weather data retrieved successfully
Rome
Accomodation data retrieved successfully
Weather data retrieved successfully
Madrid
Accomodation data retrieved successfully
Weather data retrieved successfully
Paris
Accomodation data retrieved successfully
Weather data retrieved successfully


  3%|▎         | 7/218 [3:15:23<99:34:10, 1698.82s/it] 

Barcelona
Accomodation data retrieved successfully
Weather data retrieved successfully
Rome
Accomodation data retrieved successfully
Weather data retrieved successfully
Madrid
Accomodation data retrieved successfully


  3%|▎         | 7/218 [12:54:35<389:08:14, 6639.31s/it]


KeyboardInterrupt: 

In [12]:

container_name = "bdmcontainerp1"
container_client = blob_service_client.get_container_client(container_name)
metadata_list = []

for blob in container_client.list_blobs():
    blob_client = container_client.get_blob_client(blob.name)
    blob_properties = blob_client.get_blob_properties()

    metadata_list.append({
        "blob_name": blob.name,
        "size": blob.size,
        "last_modified": blob.last_modified.isoformat() if blob.last_modified else None,
        "creation_time": getattr(blob_properties, "creation_time", None),
        "etag": blob.etag,
        "content_type": blob_properties.content_settings.content_type,
        "content_encoding": blob_properties.content_settings.content_encoding,
        "content_language": blob_properties.content_settings.content_language,
        "content_disposition": blob_properties.content_settings.content_disposition,
        "cache_control": blob_properties.content_settings.cache_control,
        "lease_status": getattr(blob_properties.lease, "status", None),
        "lease_state": getattr(blob_properties.lease, "state", None),
        "lease_duration": getattr(blob_properties.lease, "duration", None),
        "access_tier": getattr(blob_properties, "blob_tier", None),
        "access_tier_inferred": getattr(blob_properties, "blob_tier_inferred", None),
        "access_tier_last_modified": getattr(blob_properties, "blob_tier_last_modified", None),
        "server_encrypted": getattr(blob_properties, "server_encrypted", None),
        "encryption_key_sha256": getattr(blob_properties, "encryption_key_sha256", None),
        "customer_provided_key_sha256": getattr(blob_properties, "customer_provided_key_sha256", None),
        "tags": blob_properties.tags if blob_properties.tags else None,
        "snapshot": getattr(blob, "snapshot", None),
        "version_id": getattr(blob, "version_id", None),
        "is_current_version": getattr(blob, "is_current_version", None),
        "rehydration_status": getattr(blob_properties, "rehydration_status", None),
        "metadata": blob_properties.metadata
    })


In [13]:
metadata_list[0]

{'blob_name': 'accomodation/Barcelona/2025-03-24_2025-03-25.json',
 'size': 39155,
 'last_modified': '2025-03-24T20:18:11+00:00',
 'creation_time': datetime.datetime(2025, 3, 24, 20, 18, 11, tzinfo=datetime.timezone.utc),
 'etag': '0x8DD6B10FFBCDD81',
 'content_type': 'application/octet-stream',
 'content_encoding': None,
 'content_language': None,
 'content_disposition': None,
 'cache_control': None,
 'lease_status': 'unlocked',
 'lease_state': 'available',
 'lease_duration': None,
 'access_tier': 'Hot',
 'access_tier_inferred': True,
 'access_tier_last_modified': None,
 'server_encrypted': True,
 'encryption_key_sha256': None,
 'customer_provided_key_sha256': None,
 'tags': None,
 'snapshot': None,
 'version_id': None,
 'is_current_version': None,
 'rehydration_status': None,
 'metadata': {}}

In [ ]:
json_filename = "blob_metadata.json"
with open(json_filename, "w", encoding="utf-8") as json_file:
    json.dump(metadata_list, json_file, indent=4)

print(f"Metadata stored in {json_filename}")



# Trusted Zone
The following transformations will be done to the data:
* Schema flattening: En columna de price por ejemplo
* Schema correction: Hacer una propuesta de esquema e imponerlo en todos los documentos (esquema + data types)

* Compress images
* Check file integrity

In [6]:
container_name = "bdmcontainerp1"
container_client = blob_service_client.get_container_client(container_name)
blobs = [blob for blob in container_client.list_blobs()]

## Accomodation Data

In [69]:
from tqdm import tqdm
import os

### Step 1: Access the data

In [17]:
for blob in blobs:
    if "accomodation/" in blob.name and "json" in blob.name:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
        blob_data = blob_client.download_blob()
        content = blob_data.readall().decode('utf-8')
        data = json.loads(content)
        with open(blob.name, "w") as file:
            json.dump(data, file)

### Step 2: Data Flattening

In [30]:
def flatten_dict(d, parent_key='', sep='_'):
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

### Step 3: Schema Comparison

In [33]:
def get_schema(d, indent=0):
    schema = {}
    for key, value in d.items():
        if isinstance(value, dict):
            schema[key] = get_schema(value, indent + 2)
        elif isinstance(value, list):
            if len(value) > 0 and isinstance(value[0], dict):
                schema[key] = [get_schema(value[0], indent + 2)]
            else:
                schema[key] = [type(value[0]).__name__ if value else "unknown"]
        else:
            schema[key] = type(value).__name__
    return schema

### Step 4: Schema Enforcement

In [39]:
def enforce_schema(data, schema):
    def cast_value(value, expected_type):
        try:
            if expected_type == "int":
                return int(value)
            elif expected_type == "float":
                return float(value)
            elif expected_type == "str":
                return str(value)
            elif expected_type == "bool":
                if isinstance(value, str):
                    return value.lower() in ["true", "1", "yes"]
                return bool(value)
            else:
                return None
        except Exception:
            return None

    def enforce(data, schema):
        if not isinstance(data, dict):
            return {}

        for key, expected in schema.items():
            if key not in data:
                data[key] = None
            else:
                if isinstance(expected, dict):
                    if not isinstance(data[key], dict):
                        data[key] = {}
                    data[key] = enforce(data[key], expected)
                elif isinstance(expected, list) and expected and isinstance(expected[0], dict):
                    if not isinstance(data[key], list):
                        data[key] = []
                    else:
                        data[key] = [enforce(item, expected[0]) if isinstance(item, dict) else {} for item in data[key]]
                elif isinstance(expected, list):
                    # Primitive lists (e.g., ["int"])
                    if not isinstance(data[key], list):
                        data[key] = []
                    else:
                        data[key] = [cast_value(item, expected[0]) for item in data[key]]
                else:
                    # Primitive value
                    data[key] = cast_value(data[key], expected)
        return data

    return enforce(data.copy(), schema)


### Step 5: Running the pipeline

In [68]:
with open("accomodation_schema.json", "r") as file:
    accomodation_schema = json.load(file)

cities = ['Barcelona', "Paris", 'Rome', 'Madrid']

for city in cities:
    for file_name in tqdm(os.listdir(f'accomodation/{city}/')):
        documents = []
        path = f'accomodation/{city}/{file_name}'
        with open(path, "r") as file:
            data = json.load(file)
        file = data['data']['hotels']
        for doc in file:
            flattened = flatten_dict(doc)
            schema = get_schema(flattened)
            standardized_data = enforce_schema(flattened, accomodation_schema)
            documents.append(standardized_data)
        path = f'trusted_accomodation/{city}/{file_name}'
        with open(path, "w") as file:
            json.dump(documents, file)

100%|██████████| 90/90 [00:00<00:00, 132.59it/s]


## Weather data

In [87]:
from datetime import datetime

### Step 1: Acces the data

In [71]:
for blob in tqdm(blobs):
    if "weather/" in blob.name and "json" in blob.name:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
        blob_data = blob_client.download_blob()
        content = blob_data.readall().decode('utf-8')
        data = json.loads(content)
        with open(blob.name, "w") as file:
            json.dump(data, file)

100%|██████████| 8470/8470 [00:11<00:00, 705.90it/s]  


### Step 2: Standardize timestamps

In [86]:
def standardized_hours(timestamp):
    dt = datetime.fromisoformat(timestamp)
    hour_only = dt.strftime("%H:%M")
    return hour_only

### Step 3: Run pipeline

In [134]:
with open("weather_schema.json", "r") as file:
    weather_schema = json.load(file)

cities = ['Barcelona', "Paris", 'Rome', 'Madrid']

for city in cities:
    for file_name in tqdm(os.listdir(f'weather/{city}/')):
        
        path = f'weather/{city}/{file_name}'
        with open(path, "r") as file:
            data = json.load(file)
        
        standardized = data['hourly']
        standardized['time'] = list(map(standardized_hours, standardized['time']))
        schema = get_schema(standardized)
        standardized_data = enforce_schema(standardized, weather_schema)
        
        path = f'trusted_weather/{city}/{file_name}'
        with open(path, "w") as file:
            json.dump(standardized_data, file)

100%|██████████| 94/94 [00:00<00:00, 1058.38it/s]


# Exploitation Zone
Create the following data artifacts:
In HDFS implement:
* Hotel
* Sunny_Dates
* Rainy_Dates
* Cloudy_Dates
* Snowy_Dates
* Weather_Ranking_Dates
* Accomodation_Ranking_Dates

In Neo4J implement:
* Points of Interest

# Transportation

Find out more in: https://openrouteservice.org/dev/#/api-docs/optimization/post
Alternatives: https://github.com/graphhopper/graphhopper/blob/master/README.md#Map-Matching
https://github.com/VROOM-Project/vroom/blob/master/docs/API.md